<a href="https://colab.research.google.com/github/RockhoRockho/Scraping-Crawling/blob/main/%EA%B5%AC%EA%B8%80_%EC%9D%B4%EB%AF%B8%EC%A7%80_%EC%8A%A4%ED%81%AC%EB%9E%98%ED%95%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 구글 이미지 스크래핑

## Selenium 및 웹 드라이버 설치

In [21]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/graph

## 라이브러리 import

In [22]:
import os
import time
import socket

from urllib.request import urlretrieve
from urllib.error import HTTPError, URLError
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException, ElementNotInteractableException
from PIL import Image

## `scroll_down()`: 스크롤을 내리는 함수

In [23]:
def scroll_down():
    scroll_count = 0
    print("[scroll_down(): 스크롤 다운 시작]")

    last_height = wd.execute_script("return document.body.scrollHeight")
    after_click = False

    while True:
        print(f"[스크롤 다운: {scroll_count}]")
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        scroll_count += 1
        time.sleep(1)

        new_height = wd.execute_script("return document.body.scrollHeight")

        if last_height == new_height:
            if after_click is True:
                break
            
            else:
                try:
                    more_button = wd.find_element_by_xpath('//*[@id="islmp"]/div/div/div/div[1]/div[2]/div[2]/input')
                    if more_button.is_displayed():
                        more_button.click()
                        after_click = True
                except NoSuchElementException as e:
                    print(e)
                    break

        last_height = new_height


## `click_and_save()`: 썸네일을 이미지 선택 후 원본 이미지 저장

In [24]:
def click_and_save(dir_name, index, img, img_list_length):
    global scraped_count

    try:
        img.click()
        wd.implicitly_wait(8)
        src = wd.find_element_by_xpath('//*[@id="Sva75c"]/div/div/div[3]/div[2]/c-wiz/div/div[1]/div[1]/div[2]/div/a/img').get_attribute('src')
        if src.split('.')[-1] == 'png':
            urlretrieve(src, dir_name + '/' + str(scraped_count + 1) + '.png')
            print(f" {index+1}/{img_list_length} PNG 이미지 저장")
        else:
            urlretrieve(src, dir_name + '/' + str(scraped_count + 1) + '.jpg')
            print(f" {index+1}/{img_list_length} JPG 이미지 저장")

        scraped_count += 1

    except HTTPError as e:
        print(e)
        pass

## `scraping()`: 구글 이미지 스크래핑 함수

In [25]:
def scraping(dir_name, query):
    global scraped_count

    url = f"https://www.google.com/search?q={query}&tbm=isch&hl=ko&tbs=isz:l&sa=X&ved=0CAIQpwVqFwoTCIis_vK8zvMCFQAAAAAdAAAAABAC&biw=1684&bih=812"
    wd.get(url)
    wd.maximize_window()

    scroll_down()

    div = wd.find_element_by_xpath('//*[@id="islrg"]/div[1]')
    img_list = div.find_elements_by_css_selector('div.bRMDJf.islir > img')

    for index, img in enumerate(img_list):
        try:
            click_and_save(dir_name, index, img, len(img_list))
        except ElementClickInterceptedException as e:
            print(e)
            wd.execute_script('window.scrollTo(0, window.scrollY + 100')
            time.sleep(1)
            click_and_save(dir_name, index, img, len(img_list))
        except NoSuchElementException as e:
            print(e)
            wd.execute_script('window.scrollTo(0, window.scrollY + 100')
            time.sleep(1)
            click_and_save(dir_name, index, img, len(img_list))
        except ConnectionResetError as e:
            print(e)
            pass
        except URLError as e:
            print(e)
            pass
        except socket.timeout as e:
            print(e)
            pass
        except socket.gaierror as e:
            print(e)
            pass
        except ElementNotInteractableException as e:
            print(e)
            break

    try:
        print("[스크래핑 종료 (성공률: %.2f%%)]" % (scraped_count / len(img_list) * 100.0))

    except ZeroDivisionError as e:
        print(e)

    wd.quit()

## `filter_and_remove()`: 일정 해상도 이하이거나 손상된 이미지 제거

In [26]:
def filter_and_remove(dir_name, query, filter_size):
    filtered_count = 0

    for index, file_name in enumerate(os.listdir(dir_name)):
        try:
            file_path = os.path.join(dir_name, file_name)
            img = Image.open(file_path)

            if img.width < filter_size and img.height < filter_size:
                img.close()
                os.remove(file_path)
                print(f"{index} 이미지 제거")
                filtered_count += 1
        except OSError as e:
            print(e)
            os.remove(file_path)
            filtered_count += 1

    print(f"[이미지 제거 개수: {filtered_count}/{scraped_count}")

## 스크래핑 실행

In [27]:
socket.setdefaulttimeout(30)

wd = webdriver.Chrome('chromedriver', options=chrome_options)
scraped_count = 0

path = './'
query = input('검색어 입력: ')

dir_name = path + query
os.makedirs(dir_name)
print(f"[{dir_name} 디렉토리 생성]")

scraping(dir_name, query)
filter_and_remove(dir_name, query, 400)

검색어 입력: sea
[./sea 디렉토리 생성]
[scroll_down(): 스크롤 다운 시작]
[스크롤 다운: 0]
[스크롤 다운: 1]
[스크롤 다운: 2]
[스크롤 다운: 3]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


[스크롤 다운: 4]
[스크롤 다운: 5]
[스크롤 다운: 6]
[스크롤 다운: 7]
[스크롤 다운: 8]
[스크롤 다운: 9]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/selenium/webdriver/remote/webelement.py:501: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  import sys


Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="Sva75c"]/div/div/div[3]/div[2]/c-wiz/div/div[1]/div[1]/div[2]/div/a/img"}
  (Session info: headless chrome=94.0.4606.71)
Stacktrace:
#0 0x562457acd6e3 <unknown>
#1 0x5624577de9f3 <unknown>
#2 0x562457814230 <unknown>
#3 0x562457847c47 <unknown>
#4 0x56245783125d <unknown>
#5 0x5624578459bc <unknown>
#6 0x562457831643 <unknown>
#7 0x5624578088ec <unknown>
#8 0x562457809db5 <unknown>
#9 0x562457af1a14 <unknown>
#10 0x562457b0115d <unknown>
#11 0x562457b00e7b <unknown>
#12 0x562457b017a2 <unknown>
#13 0x562457b3a52b <unknown>
#14 0x562457b01a01 <unknown>
#15 0x562457ae68f1 <unknown>
#16 0x562457b0a408 <unknown>
#17 0x562457b0a59a <unknown>
#18 0x562457b2432f <unknown>
#19 0x7f0540fc36db start_thread



JavascriptException: ignored

In [19]:
!ls

sample_data  sea


In [20]:
!rmdir sea